### Quantum Approach and QUBO expression
Let us represent the real case scenario of the delivery planning problem into a graph problem.

We can represent the transport network as a **graph** where the delivery locations are the vertices $v_i$, the path between two locations as an edge $e_{ij}$, the distance or delivery cost between the respective locations as the cost of the edge $C_{ij}$, the capacity of a vehicle as $W$, $m$ represent the number of vehicles carrying the load.

The aim is to minimize the **cost** of the transportation keeping in mind the given **constraints**:

1. $ 0 \leq x_i \leq W$ - The total weight of goods delivered prior to the location i should be less than W.   
2. $x_i + w_i - x_j \leq M(1-y_{ij})$  - The sum of customer demand within a route does not exceed the vehicles’ capacity
3. $\sum_{j \in OUT(i)} y_{ij} =1 $- each customer in V \ {Depot} is visited exactly once by exactly one vehicle;

where $w_i$ is the delivery volume to vertex $i$, $x_i$ is the sum delivery volume prior to vertex $i$.

We try to divide the problem into two parts in order to solve the given problem.
1. **Clustering** : We divide the graph into clusters such that each cluster has a cumulative sum of the delivery volume $ \leq W$.
2. We find the shortest routes in each cluster to minimize the overall delivery cost.

For the **clustering** part we use the famous classical **Knapsack problem** in which we have a sack with a capacity of M and a group of objects to be filled in the **sack**. The objects are arranged in groups such that weight of each group being $W_i$ and the number of objects in the respective group being $C_i$. The aim is to maximize the number of objects in the sack keeping the weight limitation in check i.e $\leq M$.

$$ W = \sum_{\alpha =1 }^n w_{\alpha} x_{\alpha} $$
where $x_i$ is either 0 or 1 depending on if the lot has been selected.

We have to maximize the cost i.e the number of objects in the sack with the given weight limit of the sack.

$$ W = \sum_{\alpha =1 }^n c_{\alpha} x_{\alpha} $$
where $c_i$ is the cost of one lot.

We take a variable $y_n$ for $ 1 \leq n \leq w$ which is 1 if the final weight of the knapsack is n and 0 otherwise.

The QUBO for the knapsack problem is:
$$ A(1-\sum_{n=1}^W y_n)^2 + A(\sum_{n=1}^W n y_n - \sum_{\alpha} w_{\alpha} x_{\alpha})^2 - B\sum_{\alpha} c_{\alpha} x_{\alpha} $$

Here the first term takes care that the weight only takes one value and the second term being the weight in the sack is the same as we claimed it to be while the third term is the cost function.


We can use the above knapsack problem for clustering in our graph. The cost of each lot which was denoted by $c_{\alpha}$ is **1** in the delivery problem as there is a single individual at each location. The difference that comes out here is the we have **m** number of vehicles that correspond to **m** number of sacks in the previous section. We modify the QUBO formulation for the clustering part based on knapsack in such a way:

$$A =  X\sum_{a=1}^m(1-\sum_{n=1}^W y_n^a)^2 + X\sum_{a=1}^{m}(\sum_{n=1}^W n y_n^a - \sum_{\alpha} w_{\alpha} x_{\alpha}^a)^2 - X\sum_{\alpha} (1-\sum_{a=1}^m \sum_{j=1}^n x_{\alpha,j}^a)^2$$

We add another term to the above QUBO format in order to group the customers close to each other together along with the knapsack approach.
$$B =  C\sum_{a=1}^m(\sum_{uv} D_{uv}x_u^a x_v^a) $$

where $D_{uv}$ is the **euclidean distance** between the vertex u and v.

The final QUBO for the clustering part is $P = A + B$.

The first part of the **A** term takes care of the constraint where each vehicle has clear capacity constraint, the second term takes care of the constraint where the capacity of the vehicle is not exceeded by the summed customer demand in one route.
The third term in **A** takes care of the fact that each delivery individual is assigned one route and it has only one position within that route.
Just like we took care of an individual being assigned to one route and one position we write another term which takes care that each position belongs to just one individual and thus one route because we are solving the clustering and routing together.

$$ C = X\sum_{j=1}^n (1-\sum_{a=1}^m  \sum_{\alpha}x_{\alpha,j}^a)^2 $$


The final term is to **optimize** the routes among each cluster **(finding shortest route in each cluster)**.

$$ D = E\sum_{j=1}^N D_{uv}x_{u,j}x_{v,j+1} $$

Thus the final QUBO comes out to be $H=A+B+C+D$ which takes care of all the constraints given above.